In [5]:
import numpy as np
a = [(np.zeros((3,2)),np.ones(5)),(np.zeros((3,2)),np.ones(5)),(np.ones((3,2)),np.ones(5)),(np.ones((3,2)),np.ones(5)),(np.zeros((3,2)),np.ones(5))]
x = [x[0].reshape((-1)) for x in a]
np.random.permutation(x)
from Gibbs import generate_sw_tuples_batch
batch = generate_sw_tuples_batch(10, p=20)
S = np.array([x[0][0] for x in batch])
w = np.array([x[1] for x in batch])
from scipy.optimize import ridder
import torch
import matplotlib.pyplot as plt
from MINE import MINE
from tqdm import tqdm
from torch.autograd import Variable
import numpy as np

import random
import os
def seed_torch(seed=1029):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
seed_torch()

model = MINE(11, 20)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
plot_loss = []
w_shuffle = np.random.permutation(w)

S_sample = Variable(torch.from_numpy(S).type(torch.FloatTensor), requires_grad=False)#训练数据都是计算图里的叶节点，不需要梯度
w_sample = Variable(torch.from_numpy(w).type(torch.FloatTensor), requires_grad=False)
w_shuffle_sample = Variable(torch.from_numpy(w_shuffle).type(torch.FloatTensor), requires_grad=False)

#自定义数值求解函数 done
def solve_f_star(y):
    if y.requires_grad:
        y_ng = y.detach() #去掉梯度信息，成为普通tensor（这一步可以删掉）
    x = []
    for y_ in y_ng[:,0]:
        y_ = y_.item()
        def f_star(x):
            return np.log(x)-1/x+1-y_
        res = ridder(f_star,0,1 if y_<0 else np.exp(y_),xtol=1e-10)
        x.append(res)
    return torch.tensor(x, requires_grad=False)

pred_xy = model(S_sample, w_sample)
pred_x_y = model(S_sample, w_shuffle_sample)

loss1 = - torch.mean(pred_xy)
t1 = solve_f_star(pred_x_y).reshape((-1,1))
loss2 = torch.mean(t1*pred_x_y)
loss2_actual = torch.mean(t1-1+torch.log(t1))
'''
plot_loss.append(loss1.data.numpy())
print(loss1.item())
'''
loss = loss1 + loss2
estimation = (-loss1 - loss2_actual).data.numpy()
model.zero_grad()
loss.backward()
optimizer.step()

array([[1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0.]])